In [25]:
import os
from datetime import datetime
from tqdm import tqdm
import pandas as pd

In [26]:
retorno = open('../files/ret/modelo.RET').readlines()


### Resultado retorno

In [27]:
def get_registro(line):
    processamento = {
        '01': 'Controlador não cadastrado no Gestao Ágil.',
        '02': 'Tipo de registro inválido.',
        '03': 'MCI(identificação interna no cadastro do BB) do controlador no registro 0 diverge MCI controlador registro 1.',
        '04': 'CNPJ jurisdicionado inválido!',
        '05': 'CNPJ não localizado no Banco do Brasil S/A',
        '06': 'Código de situação inválido.',
        '07': 'Inativação de CNPJ jurisdicionado inexistente no GFA.',
        '08': 'UF do CNPJ jurisdicionado não autorizado para o controlador.  ',
        '09': 'Município do CNPJ jurisdicionado não autorizado para o controlador.',
        '10': 'Natureza jurídica do CNPJ jurisdicionado não autorizada para o controlador.',
        '11': 'Quantidade registros "detalhe" informada no trailer inválida ou divergente da quantidade  detalhes lidos.',
        '12': 'Cadastro desatualizado impede validação de UF.'
    }

    result = {
        "mci": line[1:10],
        "cnpj": line[10:24],
        "situacao": line[24],
        "processamento": line[25:27],
        "processamento_status": processamento[line[25:27]]
    }

    return result


In [28]:
registros = {}
for line in tqdm(retorno[1:-1]):
    ret = get_registro(line)
    registros[ret['cnpj']] = ret

100%|██████████| 4/4 [00:00<00:00, 14157.99it/s]


In [29]:
df = pd.DataFrame(registros).T
df

,mci,cnpj,situacao,processamento,processamento_status
12345678198401,000000000,12345678198401,S,05,CNPJ não localizado no Banco do Brasil S/A
12345678198402,000000000,12345678198402,S,05,CNPJ não localizado no Banco do Brasil S/A
12345678198403,000000000,12345678198403,S,08,UF do CNPJ jurisdicionado não autorizado para ...
12345678198404,000000000,12345678198404,S,10,Natureza jurídica do CNPJ jurisdicionado não a...


In [30]:
df['processamento_status'].value_counts()

processamento_status
CNPJ não localizado no Banco do Brasil S/A                                     2
UF do CNPJ jurisdicionado não autorizado para o controlador.                   1
Natureza jurídica do CNPJ jurisdicionado não autorizada para o controlador.    1
Name: count, dtype: int64

### salvar status retorno

In [40]:
df.to_csv('../files/ret/retorno.csv')

### Remessa x Retorno

In [43]:
cnpjs_erro = df['cnpj'].to_list()

In [44]:
cnpjs_erro

['12345678198401', '12345678198402', '12345678198403', '12345678198404']

In [45]:
len('1000000000')

10

In [47]:
remessa = open('../files/remessa.txt').readlines()
cnpjs_enviados = [registro[10:24] for registro in remessa[1:-1]]
cnpjs_enviados

['12345678198401',
 '12345678198402',
 '12345678198403',
 '12345678198404',
 '12345678198405']

In [48]:
cnpjs_processados = set(cnpjs_enviados).difference(set(cnpjs_erro))
cnpjs_processados

{'12345678198405'}